# Gaussian Process Regression and Bayesian Global Optimisation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import gp as gp
import kernels as kernels
import acquisitions as acquisitions 
import bayesopt as bayesopt
import importlib

importlib.reload(gp)
importlib.reload(kernels)
importlib.reload(acquisitions)
importlib.reload(bayesopt)
%matplotlib inline

plt.rcParams['xtick.major.pad']='2'
plt.rcParams['ytick.major.pad']='2'


#### We want to maximise the following 1-dimensional function,
$$\mathrm{maximise}_x\quad f(x)$$
where
$$f(x) = \sin(x) + \sin(2x) + \epsilon$$


In [ ]:
bounds = np.array([[-3, 6]])
noise_level = 0.1

np.random.seed(4)

def f(X, noise_level=noise_level):
    return np.sin(X) + np.sin(2 * X) + noise_level * np.random.randn(*X.shape)

X_init = np.array([[-0.5], [2.2]])
Y_init = f(X_init)

# Bound our random variable X
X = np.arange(bounds[:, 0], bounds[:, 1], 0.02).reshape(-1, 1)

# First let's have a noise-free objective function
Y = f(X, 0)

#Plot optimisation objective with the appropriate noise levels
plt.figure(figsize=(20, 10))
plt.plot(X, Y, 'r--', lw=2, label='Noise-free objective');
plt.plot(X, f(X), 'bx', lw=1, alpha=0.6, label='Noisy samples');
plt.plot(X_init, Y_init, 'kx', mew=3, label='Initial samples');
plt.title("Expensive Black-Box Function", fontdict = {'fontsize' : 20})
plt.legend()
plt.show()


In [ ]:
# Expected Output:
# 1) An animation of the Bayesian optimisation process which will lead to the close approximation of the surrogate function to the black box one.
# 2) An image of the final iteration (result) of the BO process (expected to be at a converged state)
# 3) Two plots indicating extra convergence heuristics

# Probability Improvement (PI)
bayesopt.BO(X_init, Y_init, f, noise_level, bounds, n_iter=12, xi=0.2, title='Bayesian Optimisation - with Probability Improvement', X=X, Y=Y).call_animation(acquisitions.probability_improvement)

In [ ]:
# Expected Output: Similar as above

# Expected Improvement (EI)
bayesopt.BO(X_init, Y_init, f, noise_level, bounds, n_iter=12, xi=0.05, title='Bayesian Optimisation - with Expected Improvement', X=X, Y=Y).call_animation(acquisitions.expected_improvement)

### Higher Dimensions

Let's assume that we want to approximate a higher dimensional function via the surrogate model, using a Gaussian Process regression.

We have the function of a N-dimensional dataset $X$, 
$$
f(X) = \tanh^2\left(\frac{||X||_F}{2}\right) + \boldsymbol{\epsilon}
$$

where we take the Frobenius norm of $X$ per column (second dimension).

Let's now investigate the benefits of performing hyper-parameter optimisation on the Gaussian process regression model during the fitting process by maximising the log marginal likelihood, before performing tasks such as Bayesian optimisation to find global optima.

In [ ]:
def plot_gp_2D(gx, gy, mu, X_train, Y_train, title, i):
    ax = plt.gcf().add_subplot(1, 3, i, projection='3d')
    ax.plot_surface(gx, gy, mu.reshape(gx.shape), cmap=cm.winter, linewidth=0.5, alpha=0.3, antialiased=False)
    ax.scatter(X_train[:,0], X_train[0:,1], Y_train, c=Y_train, cmap=cm.autumn)
    ax.set_title(title)

noise_2D = 0.1
np.random.seed(3408944656)

def f2D(X, noise_2D=noise_2D):
    return np.tanh(0.5 * np.linalg.norm(X, axis=1))**2 + noise_2D * np.random.randn(len(X))


rx, ry = np.arange(-10, 10, 0.1), np.arange(-10, 10, 0.1)
gx, gy = np.meshgrid(rx, rx)
X_2D = np.c_[gx.ravel(), gy.ravel()]

Y_1D = f2D(X_2D, 0)

X_2D_train = np.random.uniform(-8, 8, (100, 2))

Y_1D_train = f2D(X_2D_train)

# TODO Q2.8b
#-------------------------------------------------------------
# FIXME
nu_s = []
#------------------------------------------------------------

for nu in nu_s:

    m = kernels.Matern(nu=nu, length_scale=1.0, variance=1.0)
    gpr = gp.GPR(kernel=m, noise_level=noise_2D**2, n_restarts=20)

    plt.figure(figsize=(20,10))
    gpr.X_train = X_2D_train
    gpr.y_train = Y_1D_train
    mu_s = gpr.predict(X_2D)
    plot_gp_2D(gx, gy, mu_s, X_2D_train, Y_1D_train, r'Before hyper-parameter optimisation: $\ell$=1.00, $\sigma^2_f$=1.00', 1)

    gpr.update(X_2D_train, Y_1D_train)
    params = gpr.kernel.get_hyperparameters()
    mu_s = gpr.predict(X_2D)
    print(f'Running a Gaussian process with a Matern class covariance function with value nu = {nu}')
    for key in sorted(params): print(f"{key} : {params[key]}")

    plot_gp_2D(gx, gy, mu_s, X_2D_train, Y_1D_train,
            r'After hyper-parameter optimisation: $\ell$={:.2f}, $\sigma^2_f$={:.2f}'.format(params["length_scale"], params["variance"]), 2)

    plot_gp_2D(gx, gy, Y_1D, X_2D_train, Y_1D_train, f'Original function without noise', 3)
    plt.show()
    print('-------------------------------------------------------------------------------------------------------------------------')

# Expected output per loop/row:
# Three 3D plots: GP approximation with default parameters (left), GP approximation with learned parameters (middle), and original function to be approximated (right)